# Hero of Pymoli Data Analysis

<1. Gender>
- A 84% majority of players are male with just 14% female. Even though there's far fewer female player, their average spend is actually ~10% higher than for males.

<2. Age Demographics>
- Outside from the two main age demographics of 15-19 and 20-24, the other age groups still spend about the same on average. In fact, the two highest spending groups are actually 30-34 (avg total $ 4.76) and <10 (avg total$ 4.54). 

<3. Items>
- Purchases are evenly spread out among many people. The top spender only purchased 5 items(total $ 18). Also, for items, the most popular item was purchased only 12 times. 4/5 most people items are also the most profitable.

In [1]:
# Dependencies
import pandas as pd
import numpy as np

In [2]:
# File to Load 
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head(2)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56


In [3]:
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

## Player Count

In [4]:
# Display the total number of players
totalPlayers = purchase_data['SN'].nunique()

totalPlayers_table = pd.DataFrame({"Total Players":[totalPlayers]})
totalPlayers_table

,Total Players
0,576


In [5]:
# Memo: This cell's result is same as the above cell

# Display the total number of players
# Memo: The value_counts method counts unique value in a column
## totalPlayers = purchase_data["SN"].value_counts()

# Memo: To print a single scalar value, wrap with []. e.g. [singleValue]
## totalPlayers_table = pd.DataFrame({"Total Players": [totalPlayers.count()]})
## totalPlayers_table.head()

## Purchasing Analysis (Total) 

In [6]:
# Number of Unique Items
uniqueItems = purchase_data["Item ID"].value_counts().count()
# Average Purchase Price
averagePurchase = purchase_data["Price"].mean()
# Format two places of decimals
averagePurchase = "{0:0.2f}".format(averagePurchase)
# Total Number of Purchases
totalPurchase = purchase_data["Price"].count()
# Total Revenue
totalRevenue = purchase_data["Price"].sum()

# Format a number to commas to separate thousands 
totalRevenue = "{:,}".format(totalRevenue)
    
purchasingSummary_table = pd.DataFrame({"Number of Unique Item":[uniqueItems], 
                                       "Average Price":"$"+str(averagePurchase), "Number of Purchases": totalPurchase, 
                                       "Total Revenue":"$"+str(totalRevenue)})
purchasingSummary_table

,Number of Unique Item,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics 

In [7]:
# Count of players per gender
# Memo: nunique = Return number of unique elements in the object
genderCounts = purchase_data.groupby('Gender')['SN'].nunique()
genderCounts = genderCounts.sort_values(ascending=False)

In [8]:
# Percentage of players per gender
genderDemographics_table = pd.DataFrame({"Gender":genderCounts, 
                                        "Percentage of Players":genderCounts / totalPlayers * 100})

genderDemographics_table.style.format({'Percentage of Players':"{0:0.2f}"})

,Gender,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


## Purchasing Analysis (Gender)

In [9]:
# Purchase Count
countPurchase = purchase_data.groupby('Gender')['Purchase ID'].nunique()
# Avg Purchase Value
avgPurchase = purchase_data.groupby('Gender')['Price'].mean()
# Total Purchase Value 
totalPurchase = purchase_data.groupby('Gender')['Price'].sum()
# Avg Total Purchase per Person
avgPurchasePerson = totalPurchase / genderCounts 

# Create a summary data frame to hold the results
purchasingAnalysis_table = pd.DataFrame({"Purchase Count":countPurchase, 
                                        "Avg Purchase Value":avgPurchase, "Total Purchase Value":totalPurchase, 
                                        "Avg Total Purchase per Person":avgPurchasePerson})
purchasingAnalysis_table

# Data Formatting with currency style
# Memo: https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
purchasingAnalysis_table.style.format({'Avg Purchase Value':"${0:0.2f}", 
                                       'Total Purchase Value':"${:,.2f}", 
                                       'Avg Total Purchase per Person':"${0:0.2f}"})

,Purchase Count,Avg Purchase Value,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [10]:
# Figure out the min and max age for binning bellow
print(purchase_data["Age"].max())
print(purchase_data["Age"].min())

45
7


## Age Demographics 

In [11]:
# Create bins in which to place values based upon Age
binsAge_list = [0, 9, 14, 19, 24, 29, 34, 39, 45]
# Create labels for these bins
binAge_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Slice the data and place it into bins and check the slicer
pd.cut(purchase_data["Age"], binsAge_list, labels=binAge_labels).head()

0    20-24
1      40+
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [12]:
# Place the data series(bin) into a new column inside of the DataFrame
purchase_data["Age Group"] = pd.cut(purchase_data["Age"], binsAge_list, labels=binAge_labels)
# Check Age group
purchase_data.head(2)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+


In [13]:
# Create a groupby object based upon "Age Group"
groupAge = purchase_data.groupby("Age Group")
# Total  Count
countTotal_age = groupAge["SN"].nunique()
# Percentage of Players
percentagePlayers_age = countTotal_age / totalPlayers * 100

#Create a summary data frame to hold the results
ageDemographics_table = pd.DataFrame({"Total Count":countTotal_age, 
                                     "Percentage of Players":percentagePlayers_age})

ageDemographics_table.style.format({'Percentage of Players':"{0:0.2f}"})

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age) 

In [14]:
# Create a group based off of the bins
groupAge_df = purchase_data.groupby("Age Group")

# Purchase Count
countPurchase_age = groupAge_df["Purchase ID"].count()
# Avg Purchase Price
avgPurchase_age = groupAge_df["Price"].mean()
# Total Purchase Value
totalPurchase_age = groupAge_df["Price"].sum()
# Avg Total Purchase per Person
avgPurchasePerson_age = totalPurchase_age / countTotal_age

# Create a summary data frame to hold the results
purchaseAnalysis_table = pd.DataFrame({"Purchase Count":countPurchase_age, 
                                     "Avg Purchase Price":avgPurchase_age, 
                                     "Total Purchase Value":totalPurchase_age, 
                                     "Avg Total Purchase per Person":avgPurchasePerson_age})

# Data Formatting with currency style
purchaseAnalysis_table.style.format({'Avg Purchase Price':"${0:0.2f}", 'Total Purchase Value':"${:,.2f}", 
                                     'Avg Total Purchase per Person':"${0:0.2f}"})

,Purchase Count,Avg Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

In [15]:
groupSN_df = purchase_data.groupby("SN")
# Purchase Count
countPurchase_SN = groupSN_df["Purchase ID"].count()
# Avg Purchase Price
avgPurchase_SN = groupSN_df["Price"].mean()
# Total Purchase Value
totalPurchase_SN = groupSN_df["Price"].sum()

topSpenders_table = pd.DataFrame({"Purchase Count":countPurchase_SN, 
                                     "Avg Purchase Price":avgPurchase_SN, 
                                     "Total Purchase Value":totalPurchase_SN})

# Sort the total purchase value column in descending order

topSpenders_sort = topSpenders_table.sort_values(["Total Purchase Value"], ascending=False).head()

# Data Formatting with currency style
# Memo: Manipulate data first and format last
# Memo: Be careful not to reassign style.format to the table object
topSpenders_sort.style.format({'Avg Purchase Price':"${0:0.2f}", 
                                                            'Total Purchase Value':"${:,.2f}"})

,Purchase Count,Avg Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items 

In [16]:
#Get the Item ID, Item Name, and Item Price columns
# items = purchase_data.loc[:, ["Item ID", "Item Name", "Price"]]
items = purchase_data[["Item ID", "Item Name", "Price"]]

#Group by Item ID and Item Name
groupItems_data = items.groupby(['Item ID', 'Item Name'])

# Purchase Count
countPurchase_items = groupItems_data["Item ID"].count()

# Total Purchase Value
totalPurchase_items = groupItems_data["Price"].sum()

# Item Price
price_items = totalPurchase_items / countPurchase_items

#Create a summary data frame to hold the results
MostPopularItems_table = pd.DataFrame({"Purchase Count":countPurchase_items, 
                                      "Item Price":price_items, "Total Purchase Value":totalPurchase_items})

#Sort the purchase count column in descending order
popularItems_sort= MostPopularItems_table.sort_values(["Purchase Count"], ascending=False).head()

# Data Formatting with currency style
popularItems_sort.style.format({'Item Price':"${0:0.2f}", 
                                                           'Total Purchase Value':"${0:0.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items 

In [17]:
# Sort the above table by total purchase value in descending order
totalPurchase_sort = MostPopularItems_table.sort_values(["Total Purchase Value"], ascending=False).head()
totalPurchase_sort
# Data Formatting with currency style
totalPurchase_sort.style.format({'Item Price':"${0:0.2f}", 
                                                           'Total Purchase Value':"${0:0.2f}"})

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
